# Extraccion de datos de la API Financial data for every need

In [28]:
#Importacion de librerias
 
import requests
from pprint import pprint
import psycopg2
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import pandas
from utils_apis import * 
import pandas as pd
from deltalake import DeltaTable
import pyarrow as pa
import os
from dotenv import load_dotenv




In [29]:
base_url = 'https://financialmodelingprep.com/api/v3'


Conectarme a la base de datos donde tengo alojado la fecha de ultima extraccion

In [30]:
# OBTINE LA ULTIMA FECHA DE EXTRACCION
ultimaExtraccion= conexion_consultas(
    'pipeline.conf', 'postgresql','postgresql+psycopg2'
    )
# OBTENER LA SESSION
session= devolver_session( 'pipeline.conf', 'postgresql','postgresql+psycopg2')

print(ultimaExtraccion)
print(session)


2024-11-25 20:00:00


In [31]:
load_dotenv(dotenv_path='.env')
api_key = os.getenv("API_KEY")

EXTRACCION FULL

In [32]:
#https://financialmodelingprep.com/api/v3/etf/list?apikey=hiskvV2OvHYzOPvYgFhgb0JqXfk67SHy
parametros = {
'apikey': api_key
} 
endpoint2=f'etf/list'
data1=obtener_data(base_url,endpoint2,parametros)
print(data1)

[{'symbol': 'AHYE.PA', 'name': 'Amundi Index Solutions - Amundi EURO High Yield Bond ESG', 'price': 249.4489, 'exchange': 'Paris', 'exchangeShortName': 'EURONEXT', 'type': 'etf'}, {'symbol': 'NDAA', 'name': 'Ned Davis Research 360 Dynamic Allocation ETF', 'price': 20.2261, 'exchange': 'NASDAQ Global Market', 'exchangeShortName': 'NASDAQ', 'type': 'etf'}, {'symbol': 'CRED', 'name': 'Columbia ETF Trust I - Columbia Research Enhanced Real Estate ETF', 'price': 23.9212, 'exchange': 'New York Stock Exchange Arca', 'exchangeShortName': 'AMEX', 'type': 'etf'}, {'symbol': 'GOVI', 'name': 'Invesco Equal Weight 0-30 Years Treasury ETF', 'price': 27.9, 'exchange': 'NASDAQ Global Market', 'exchangeShortName': 'NASDAQ', 'type': 'etf'}, {'symbol': 'RSPT', 'name': 'Invesco S&P 500 Equal Weight Technology ETF', 'price': 39.08, 'exchange': 'New York Stock Exchange', 'exchangeShortName': 'NYSE', 'type': 'etf'}, {'symbol': 'DXJ', 'name': 'WisdomTree Japan Hedged Equity Fund', 'price': 110.32, 'exchange':

In [33]:
df_full= construir_tabla(data1)
df_full.head(10)

,symbol,name,price,exchange,exchangeShortName,type
0,AHYE.PA,Amundi Index Solutions - Amundi EURO High Yiel...,249.4489,Paris,EURONEXT,etf
1,NDAA,Ned Davis Research 360 Dynamic Allocation ETF,20.2261,NASDAQ Global Market,NASDAQ,etf
2,CRED,Columbia ETF Trust I - Columbia Research Enhan...,23.9212,New York Stock Exchange Arca,AMEX,etf
3,GOVI,Invesco Equal Weight 0-30 Years Treasury ETF,27.9000,NASDAQ Global Market,NASDAQ,etf
4,RSPT,Invesco S&P 500 Equal Weight Technology ETF,39.0800,New York Stock Exchange,NYSE,etf
5,DXJ,WisdomTree Japan Hedged Equity Fund,110.3200,New York Stock Exchange Arca,AMEX,etf
6,HAWX,iShares Currency Hedged MSCI ACWI ex U.S. ETF,32.6800,New York Stock Exchange Arca,AMEX,etf
7,CLOU,Global X Cloud Computing ETF,24.8700,NASDAQ Global Market,NASDAQ,etf
8,KOCT,Innovator U.S. Small Cap Power Buffer ETF - Oc...,31.7200,New York Stock Exchange Arca,AMEX,etf
9,ONLN,ProShares Online Retail ETF,46.4000,New York Stock Exchange Arca,AMEX,etf


Guardamos la extraccion full en la primera etapa bronze

In [34]:
path1= './delta_Extraccion_full/bronze1'
extraccion_full= guardar_Extraccion_full(df_full, path1, partition_cols=None)

Transformaciones de la tabla df_full

In [35]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12429 entries, 0 to 12428
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             12429 non-null  object 
 1   name               12427 non-null  object 
 2   price              12405 non-null  float64
 3   exchange           12428 non-null  object 
 4   exchangeShortName  12428 non-null  object 
 5   type               12429 non-null  object 
dtypes: float64(1), object(5)
memory usage: 582.7+ KB


In [36]:
# verificamos si existen valores nulos
df_full.isnull().sum()

symbol                0
name                  2
price                24
exchange              1
exchangeShortName     1
type                  0
dtype: int64

In [37]:
# Agregamos un 0 a los valores nulos
df_sin_null= df_full.fillna(0 , inplace=False)

In [38]:
df_sin_null.isnull().sum()

symbol               0
name                 0
price                0
exchange             0
exchangeShortName    0
type                 0
dtype: int64

In [39]:
#print(df_sin_null['price'].dtype)
print(df_sin_null.dtypes)

symbol                object
name                  object
price                float64
exchange              object
exchangeShortName     object
type                  object
dtype: object


In [40]:
# obtener el tipo de dato de cada columna
print(df_sin_null.apply(lambda col: type(col.iloc[0])))


symbol                         <class 'str'>
name                           <class 'str'>
price                <class 'numpy.float64'>
exchange                       <class 'str'>
exchangeShortName              <class 'str'>
type                           <class 'str'>
dtype: object


In [41]:

df_sin_null['ID']= df_sin_null.index



In [42]:
# Crear una columna rank que enumere secuencialmente dentro de cada grupo
df_sin_null["rank"] = df_sin_null.groupby("exchangeShortName").cumcount() + 1

df_sin_null = df_sin_null.sort_values(by=["exchangeShortName", "rank"])

# Verificar el resultado
df_sin_null.head(60)




,symbol,name,price,exchange,exchangeShortName,type,ID,rank
701,CSJP,iShares MSCI Japan ETF USD Acc,0.0000,0,0,etf,701,1
2,CRED,Columbia ETF Trust I - Columbia Research Enhan...,23.9212,New York Stock Exchange Arca,AMEX,etf,2,1
5,DXJ,WisdomTree Japan Hedged Equity Fund,110.3200,New York Stock Exchange Arca,AMEX,etf,5,2
6,HAWX,iShares Currency Hedged MSCI ACWI ex U.S. ETF,32.6800,New York Stock Exchange Arca,AMEX,etf,6,3
8,KOCT,Innovator U.S. Small Cap Power Buffer ETF - Oc...,31.7200,New York Stock Exchange Arca,AMEX,etf,8,4
9,ONLN,ProShares Online Retail ETF,46.4000,New York Stock Exchange Arca,AMEX,etf,9,5
11,GRNB,VanEck Green Bond ETF,24.0685,New York Stock Exchange Arca,AMEX,etf,11,6
12,SPMV,Invesco S&P 500 Minimum Variance ETF,47.7420,New York Stock Exchange Arca,AMEX,etf,12,7
14,SCHA,Schwab U.S. Small-Cap ETF,28.3200,New York Stock Exchange Arca,AMEX,etf,14,8
15,FLSP,Franklin Systematic Style Premia ETF,24.0250,New York Stock Exchange Arca,AMEX,etf,15,9


In [43]:
df_sin_null['exchange'].count()

12429

EXTRACCION INCREMENTAL STATEFUL

se extrae de: API de criptomonedas intradía(Proporciona datos de precios intradiarios de todas las criptomonedas que se comercializan en bolsas de todo el mundo.)


In [44]:
""" 
Bloque donde se hace se arma la URL para hacer una peticion GET a la API,
y luego se pasa los parametros a la funcion get_data para convertir los datos obtenidos
en un formato JSON
"""
from datetime import datetime, timezone,timedelta
"""
se convierte la ultima fecha de extraccion a una zona horaria para poder restar con la fecha actual(la actual estaba adelantada)
y lograr traer desde la api solo las fechas posteriores a la ultima extraccion.
"""
ultimaExtraccion = ultimaExtraccion.astimezone(timezone.utc)
#para las consultas a la api solo necesito la fecha, asi que obtengo con el metodo date()
solo_ultima_fecha = ultimaExtraccion.date()


fecha_extraccion = (datetime.now(timezone.utc))
solo_fecha_extraccion = fecha_extraccion.date()



endpoint1='historical-chart'
timeframe = "1hour"
symbol = "ALIENUSD"
#symbol = "BTCUSD"
#from_date= date_now - relativedelta(months=1)

params = {
    "from": solo_ultima_fecha ,
    "to": solo_fecha_extraccion, 
    "apikey": api_key
  
}
endpoint = f"{endpoint1}/{timeframe}/{symbol}"
data = obtener_data(base_url, endpoint, params=params)

if data:
    # Obtener la fecha máxima de los datos recibidos
    fechas_datos = [(d['date']) for d in data if 'date' in d]
    if fechas_datos:
        fecha_maxima_datos = max(fechas_datos)
        print("Fecha máxima:", fecha_maxima_datos)

        # Actualizar la fecha de extracción con la fecha máxima obtenida
        actualizar_fecha_y_proceso(session, fecha_maxima_datos, 'extraccion incremental')
    else:
        print("No se encontraron datos en la respuesta de la API.")
else:
    print("No se obtuvieron datos de la API.")


print(data)

Fecha máxima: 2024-11-25 20:00:00
Fecha y proceso actualizados correctamente.
[{'date': '2024-11-25 20:00:00', 'open': 0.00399369, 'low': 0.00399369, 'high': 0.00399369, 'close': 0.00399369, 'volume': 0.24374999999999994}, {'date': '2024-11-25 19:00:00', 'open': 0.00400947, 'low': 0.00399369, 'high': 0.00400947, 'close': 0.00399369, 'volume': 0.22499999999999995}, {'date': '2024-11-25 18:00:00', 'open': 0.00393971, 'low': 0.00393971, 'high': 0.00400947, 'close': 0.00400947, 'volume': 0.22499999999999995}, {'date': '2024-11-25 17:00:00', 'open': 0.00399199, 'low': 0.00393971, 'high': 0.00399199, 'close': 0.00393971, 'volume': 0.22499999999999995}, {'date': '2024-11-25 16:00:00', 'open': 0.00399199, 'low': 0.00399199, 'high': 0.00399199, 'close': 0.00399199, 'volume': 0.22499999999999995}, {'date': '2024-11-25 15:00:00', 'open': 0.00398904, 'low': 0.00398904, 'high': 0.00399199, 'close': 0.00399199, 'volume': 0.22499999999999995}, {'date': '2024-11-25 14:00:00', 'open': 0.00398904, 'low'

In [45]:
"""
Llamo a la funcion build_table para convertir el JSON en un DataFrame,
 y coloco un limite de salida para mostrar solo 20 registros del DF
"""
df1= construir_tabla(data,sort_column='date', ascending=True)
pd.set_option('display.max_rows', None) # sirve para mostrar todas las filas completas sin que acortar con ...


df1


,date,open,low,high,close,volume
21,2024-11-25 00:00:00,0.003909,0.003900,0.003909,0.003900,0.000000
20,2024-11-25 01:00:00,0.003900,0.003900,0.003900,0.003900,0.000000
19,2024-11-25 01:17:00,0.003900,0.003900,0.003900,0.003900,0.000000
18,2024-11-25 02:00:00,0.003900,0.003900,0.003937,0.003937,0.200000
17,2024-11-25 03:00:00,0.003937,0.003937,0.003937,0.003937,0.200000
16,2024-11-25 04:00:00,0.003937,0.003937,0.004009,0.004009,0.222917
15,2024-11-25 05:00:00,0.004009,0.004009,0.004026,0.004026,0.225000
14,2024-11-25 06:00:00,0.004026,0.004026,0.004044,0.004044,0.225000
13,2024-11-25 07:00:00,0.004044,0.004044,0.004045,0.004045,0.225000
12,2024-11-25 08:00:00,0.004045,0.004045,0.004045,0.004045,0.225000


Etapa de ingesta en la primera version: Bronze

In [46]:
"""
En esta parte se realiza una Transformacion de Estandarizacion de fechas, 
donde se convierten las fechas una zona horaria UTC y se localiza a una zona horaria UTC
para los casos donde hay fechas con localizacion none 
"""
path= './delta_table/bronze'

result= guardar_en_tabla_delta(df1,
                        path,
                        partition_cols=None)

No hay datos nuevos para ingresar, la tabla esta actualizada


In [47]:
leer_delta= DeltaTable(path)
dfnuevo= leer_delta.to_pandas()
dfnuevo

,date,open,low,high,close,volume,symbol,__index_level_0__
0,2024-11-25 20:00:00,0.003994,0.003994,0.003994,0.003994,0.112500,None,NaN
1,2024-11-25 19:00:00,0.004009,0.003994,0.004009,0.003994,0.225000,None,NaN
2,2024-11-18 17:00:00,0.003797,0.003797,0.003797,0.003797,0.000000,None,NaN
3,2024-11-18 19:00:00,0.003823,0.003823,0.003823,0.003823,0.000000,None,NaN
4,2024-11-18 21:00:00,0.003823,0.003823,0.003823,0.003823,0.000000,None,NaN
5,2024-11-19 00:00:00,0.003759,0.003759,0.003759,0.003759,0.000000,None,NaN
6,2024-11-19 04:00:00,0.003760,0.003712,0.003760,0.003712,5.000000,None,NaN
7,2024-11-19 07:00:00,0.003713,0.003713,0.003713,0.003713,0.000000,None,NaN
8,2024-11-19 10:00:00,0.003713,0.003713,0.003713,0.003713,0.000000,None,NaN
9,2024-11-19 22:00:00,0.003606,0.003606,0.003606,0.003606,0.000000,None,NaN


Realizamos algunas modificaciones a las tablas y luego guardamos en la segunda capa: Silver

In [48]:
# Agregamos una columna id y una columna de symbol que seria el nombre de la cripto


df1['symbol']= symbol
df1.reset_index(drop=True, inplace=True)
df1['id'] = df1.index + 1
df1

,date,open,low,high,close,volume,symbol,id
0,2024-11-25 00:00:00+00:00,0.003909,0.003900,0.003909,0.003900,0.000000,ALIENUSD,1
1,2024-11-25 01:00:00+00:00,0.003900,0.003900,0.003900,0.003900,0.000000,ALIENUSD,2
2,2024-11-25 01:17:00+00:00,0.003900,0.003900,0.003900,0.003900,0.000000,ALIENUSD,3
3,2024-11-25 02:00:00+00:00,0.003900,0.003900,0.003937,0.003937,0.200000,ALIENUSD,4
4,2024-11-25 03:00:00+00:00,0.003937,0.003937,0.003937,0.003937,0.200000,ALIENUSD,5
5,2024-11-25 04:00:00+00:00,0.003937,0.003937,0.004009,0.004009,0.222917,ALIENUSD,6
6,2024-11-25 05:00:00+00:00,0.004009,0.004009,0.004026,0.004026,0.225000,ALIENUSD,7
7,2024-11-25 06:00:00+00:00,0.004026,0.004026,0.004044,0.004044,0.225000,ALIENUSD,8
8,2024-11-25 07:00:00+00:00,0.004044,0.004044,0.004045,0.004045,0.225000,ALIENUSD,9
9,2024-11-25 08:00:00+00:00,0.004045,0.004045,0.004045,0.004045,0.225000,ALIENUSD,10


In [49]:
# convierto la columna symbol a un string que pandas pueda intrepretar como string, porque mas arriba ya lo pase a
#  string pero pandas no reconoce explicitamente entre object y string
df1['symbol'] = df1['symbol'].astype('string')
df1.head(10)


,date,open,low,high,close,volume,symbol,id
0,2024-11-25 00:00:00+00:00,0.003909,0.003900,0.003909,0.003900,0.000000,ALIENUSD,1
1,2024-11-25 01:00:00+00:00,0.003900,0.003900,0.003900,0.003900,0.000000,ALIENUSD,2
2,2024-11-25 01:17:00+00:00,0.003900,0.003900,0.003900,0.003900,0.000000,ALIENUSD,3
3,2024-11-25 02:00:00+00:00,0.003900,0.003900,0.003937,0.003937,0.200000,ALIENUSD,4
4,2024-11-25 03:00:00+00:00,0.003937,0.003937,0.003937,0.003937,0.200000,ALIENUSD,5
5,2024-11-25 04:00:00+00:00,0.003937,0.003937,0.004009,0.004009,0.222917,ALIENUSD,6
6,2024-11-25 05:00:00+00:00,0.004009,0.004009,0.004026,0.004026,0.225000,ALIENUSD,7
7,2024-11-25 06:00:00+00:00,0.004026,0.004026,0.004044,0.004044,0.225000,ALIENUSD,8
8,2024-11-25 07:00:00+00:00,0.004044,0.004044,0.004045,0.004045,0.225000,ALIENUSD,9
9,2024-11-25 08:00:00+00:00,0.004045,0.004045,0.004045,0.004045,0.225000,ALIENUSD,10


In [50]:
# consultar la cantidad de filas
print(f'la cantidad de fila es: {df1.shape[0]}')

la cantidad de fila es: 22


In [51]:
# ingesta silver
path= './delta_table/silver'

result= guardar_en_tabla_delta(df1,
                        path,
                        partition_cols=None)


No hay datos nuevos para ingresar, la tabla esta actualizada


In [52]:
dtable = DeltaTable(path)
d=dtable.to_pandas()
d.head(10)

,date,open,low,high,close,volume,symbol,id
0,2024-11-25 20:00:00,0.003994,0.003994,0.003994,0.003994,0.1125,ALIENUSD,22
1,2024-11-25 19:00:00,0.004009,0.003994,0.004009,0.003994,0.2250,ALIENUSD,21
2,2024-11-18 17:00:00,0.003797,0.003797,0.003797,0.003797,0.0000,ALIENUSD,20
3,2024-11-18 19:00:00,0.003823,0.003823,0.003823,0.003823,0.0000,ALIENUSD,22
4,2024-11-18 21:00:00,0.003823,0.003823,0.003823,0.003823,0.0000,ALIENUSD,24
5,2024-11-19 00:00:00,0.003759,0.003759,0.003759,0.003759,0.0000,ALIENUSD,27
6,2024-11-19 04:00:00,0.003760,0.003712,0.003760,0.003712,5.0000,ALIENUSD,33
7,2024-11-19 07:00:00,0.003713,0.003713,0.003713,0.003713,0.0000,ALIENUSD,36
8,2024-11-19 10:00:00,0.003713,0.003713,0.003713,0.003713,0.0000,ALIENUSD,39
9,2024-11-19 22:00:00,0.003606,0.003606,0.003606,0.003606,0.0000,ALIENUSD,51


In [53]:
print(f'la cantidad de fila es: {d.shape[0]}')

la cantidad de fila es: 203
